# Face verification using Siamese Networks

### Goals
- train a network for face similarity using siamese networks
- work data augmentation, generators and hard negative mining
- use the model on your picture


### Dataset

- We will be using Labeled Faces in the Wild (LFW) dataset available openly at http://vis-www.cs.umass.edu/lfw/
- For computing purposes, we'll only restrict ourselves to a subpart of the dataset. You're welcome to train on the whole dataset on GPU, by changing the PATH in the following cells, and in data download
- We will also load pretrained weights


In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
set_session(tf.Session(config=config))

In [ ]:
import keras
import os
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Input, Concatenate, merge, Lambda, Dot
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, Flatten, Dropout
import numpy as np
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

## Processing the dataset

The dataset consists of folders corresponding to each identity. The folder name is the name of the person.
We map each class (identity) to an integer id, and build mappings as dictionaries `name_to_classid` and `classid_to_name`

In [ ]:
PATH = "lfw-a/lfw/"
#PATH = "lfw/lfw-deepfunneled/"

In [ ]:
dirs = sorted(os.listdir(PATH))
name_to_classid = {d:i for i,d in enumerate(dirs)}
classid_to_name = {v:k for k,v in name_to_classid.items()}
num_classes = len(name_to_classid)
print("number of classes: "+str(num_classes))

In each directory, there is one or more images corresponding to the identity. We map each image path with an integer id, then build a few dictionaries:
- mappings from imagepath and image id: `path_to_id` and `id_to_path`
- mappings from class id to image ids: `classid_to_ids` and `id_to_classid`

In [ ]:
# read all directories
img_paths = {c:[directory + "/" + img for img in sorted(os.listdir(PATH+directory))] 
             for directory,c in name_to_classid.items()}

# retrieve all images
all_images_path = []
for img_list in img_paths.values():
    all_images_path += img_list

# map to integers
path_to_id = {v:k for k,v in enumerate(all_images_path)}
id_to_path = {v:k for k,v in path_to_id.items()}

In [ ]:
# build mappings between images and class
classid_to_ids = {k:[path_to_id[path] for path in v] for k,v in img_paths.items()}
id_to_classid = {v:c for c,imgs in classid_to_ids.items() for v in imgs}
dict(list(id_to_classid.items())[0:13])

The following histogram shows the number of images per class: there are many classes with only one image. 
These classes are useful as negatives, only as we can't make a positive pair with them.

In [ ]:
[(classid_to_name[x], len(classid_to_ids[x])) for x in np.argsort([len(v) for k,v in classid_to_ids.items()])[::-1][:10]]

In [ ]:
plt.hist([len(v) for k,v in classid_to_ids.items()], bins=range(1,10))
plt.show()

In [ ]:
[(classid_to_name[x], len(classid_to_ids[x])) for x in np.argsort([len(v) for k,v in classid_to_ids.items()])[::-1][:10]]

### Siamese nets

A siamese net takes as input two images $x_1$ and $x_2$ and outputs a single value which corresponds to the similarity between $x_1$ and $x_2$, as follows:

<img src="images/siamese.svg" style="width: 600px;" />



In order to train such a system, one has to build positive and negative pairs for the training. 

In [ ]:
# build pairs of positive image ids for a given classid
def build_pos_pairs_for_id(classid, max_num=50):
    imgs = classid_to_ids[classid]
    if len(imgs) == 1:
        return []
    pos_pairs = [(imgs[i], imgs[j]) for i in range(len(imgs)) for j in range(i+1,len(imgs))]
    random.shuffle(pos_pairs)
    return pos_pairs[:max_num]


In [ ]:
# build pairs of negative image ids for a given classid
def build_neg_pairs_for_id(classid, classes, max_num=20):
    imgs = classid_to_ids[classid]
    neg_classes_ids = random.sample(classes, max_num+1)
    if classid in neg_classes_ids:
        neg_classes_ids.remove(classid)
    neg_pairs = []
    for id2 in range(max_num):
        img1 = imgs[random.randint(0,len(imgs)-1)]
        imgs2 = classid_to_ids[neg_classes_ids[id2]]
        img2 = imgs2[random.randint(0,len(imgs2)-1)]
        neg_pairs += [(img1, img2)]
    return neg_pairs

Let's build positive and a negative pairs for class 5

In [ ]:
build_pos_pairs_for_id(5, 20)

In [ ]:
build_neg_pairs_for_id(5, list(range(num_classes)), 6)

Now that we have a way to compute the pairs, let's open all the possible images. It will expand all the images into RAM memory. There are more than 1000 images, so 100Mo of RAM will be used, which will not cause any issue.

_Note: if you plan on opening more images, you should not open them all at once, and rather build a generator_

In [ ]:
from skimage.io import imread
from skimage.transform import resize

def resize100(img):
    return resize(img, (100, 100), preserve_range=True, mode='reflect')[20:80,20:80,:]

def open_all_images(id_to_path):
    all_imgs = []
    for path in id_to_path.values():
        all_imgs += [np.expand_dims(resize100(imread(PATH+path)),0)]
    return np.vstack(all_imgs)


In [ ]:
all_imgs = open_all_images(id_to_path)
all_imgs.shape, str(all_imgs.nbytes / 1e6) + "Mo"

The following function builds a large number of positives/negatives pairs (train and test)

In [ ]:
def build_train_test_data(split=0.8):
    listX1 = []
    listX2 = []
    listY = []
    split = int(num_classes * split)
    
    # train
    for id in range(split):
        pos = build_pos_pairs_for_id(id)
        neg = build_neg_pairs_for_id(id, list(range(split)))
        for pair in pos:
            listX1 += [pair[0]]
            listX2 += [pair[1]]
            listY += [1]
        for pair in neg:
            if sum(listY) > len(listY) / 2:
                listX1 += [pair[0]]
                listX2 += [pair[1]]
                listY += [0]
    perm = np.random.permutation(len(listX1))
    X1_ids_train, X2_ids_train, Y_ids_train = np.array(listX1)[perm], np.array(listX2)[perm], np.array(listY)[perm]
    
    listX1 = []
    listX2 = []
    listY = []
    
    #test
    for id in range(split,num_classes):
        pos = build_pos_pairs_for_id(id)
        neg = build_neg_pairs_for_id(id, list(range(split,num_classes)))
        for pair in pos:
            listX1 += [pair[0]]
            listX2 += [pair[1]]
            listY += [1]
        for pair in neg:
            if sum(listY) > len(listY) / 2:
                listX1 += [pair[0]]
                listX2 += [pair[1]]
                listY += [0]
    X1_ids_test, X2_ids_test, Y_ids_test = np.array(listX1), np.array(listX2), np.array(listY)
    return X1_ids_train, X2_ids_train, Y_ids_train, X1_ids_test, X2_ids_test, Y_ids_test

In [ ]:
X1_ids_train, X2_ids_train, train_Y, X1_ids_test, X2_ids_test, test_Y = build_train_test_data()

**Data augmentation and generator**

We're building a generator, which will modify images through dataaugmentation on the fly. 
The generator enables
We use iaa library which offers tremendous possibilities for data augmentation


In [ ]:
from imgaug import augmenters as iaa

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
])


In [ ]:
class Generator():
    def __init__(self, X1, X2, Y, batch_size, all_imgs):
        self.cur_train_index=0
        self.batch_size = batch_size
        self.X1 = X1
        self.X2 = X2
        self.Y = Y
        self.imgs = all_imgs
        self.num_samples = Y.shape[0]
        
    def next_train(self):
        while 1:
            self.cur_train_index += self.batch_size
            if self.cur_train_index >= self.num_samples:
                self.cur_train_index=0
            
            imgs1 = self.X1[self.cur_train_index:self.cur_train_index+self.batch_size]
            imgs2 = self.X2[self.cur_train_index:self.cur_train_index+self.batch_size]
    
       # deactivate augmentation
       #     yield ([self.imgs[imgs1], self.imgs[imgs2]],
       #             self.Y[self.cur_train_index:self.cur_train_index+self.batch_size])
        
            yield ([seq.augment_images(self.imgs[imgs1]), 
                    seq.augment_images(self.imgs[imgs2])
                    ],
                    self.Y[self.cur_train_index:self.cur_train_index+self.batch_size]
                )

In [ ]:
gen = Generator(X1_ids_train, X2_ids_train, train_Y, 32, all_imgs)

In [ ]:
[x1, x2], y = next(gen.next_train())

In [ ]:
plt.figure(figsize=(16, 6))
for i in range(6):
    plt.subplot(2, 6, i + 1)
    plt.imshow(x1[i] / 255)
    plt.axis('off')
for i in range(6):
    plt.subplot(2, 6, i + 7)
    plt.imshow(x2[i] / 255)
    if y[i]==1.0:
        plt.title("similar")
    else:
        plt.title("different")
    plt.axis('off')
plt.show()

**Exercise**
- Add your own dataaugmentations in the process. You may look at: https://github.com/aleju/imgaug
- Be careful not to make the task to difficult, and to add meaningful augmentations
- Rerun the generator to check whether batches look correct

** Test images **

- In addition to our generator, we need test images, unaffected by the augmentation

In [ ]:
test_X1 = all_imgs[X1_ids_test]
test_X2 = all_imgs[X2_ids_test]

In [ ]:
test_X1.shape, test_X2.shape, test_Y.shape

## Simple convolutional model

In [ ]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 0.25
    
    return K.mean( y_true * K.square(1 - y_pred) +
                  (1 - y_true) * K.square(K.maximum(y_pred - margin, 0)))

In [ ]:
def accuracy_sim(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on similarity.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred > 0.5, y_true.dtype)))

**Exercise**
- Build a convolutional model which transforms the input to a fixed dimension $d = 50$
- You may use Convolutions, Maxpooling, Dropout, Dense, ...

In [ ]:
inp = Input((60,60,3), dtype='float32')

#TODO

shared_conv = Model(inputs=inp, outputs = x)

In [ ]:
# %load solutions/shared_conv.py

In [ ]:
shared_conv.summary()

**Exercise**

Build the siamese model using:
- shared_conv on both inputs
- cosine similarity
- the loss is the constrastive loss
- add as a metric the accuracy defined above
    

In [ ]:
i1 = Input((60,60,3), dtype='float32')
i2 = Input((60,60,3), dtype='float32')

# TODO

model = Model(inputs=[i1, i2], outputs=out)

In [ ]:
# %load solutions/siamese.py

We can now fit the model and checkpoint it to keep the best version 

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


best_model_fname = "siamese_checkpoint_b.h5"
best_model_cb = ModelCheckpoint(best_model_fname, monitor='val_loss',
                                save_best_only=True, verbose=1)

In [ ]:
model.fit_generator(generator=gen.next_train(), 
                    steps_per_epoch=train_Y.shape[0] // 32, 
                    epochs=5,
                    validation_data=([test_X1, test_X2], test_Y),
                    callbacks=[best_model_cb]
                   )

In [ ]:
model.load_weights("siamese_checkpoint_b.h5")

In [ ]:
# You may load a pre-trained model if you have the exact solution architecture. 
# This model is a start, but far from perfect !
# model.load_weights("pretrained_partial.h5")

**Exercise**

Finding the most similar images

- Run the shared_conv model on all images
- build a `most_sim` function which returns the most similar vectors to a given vector

In [ ]:
# TODO

emb = None

def most_sim(x, emb, topn=5):
    return None

In [ ]:
# %load solutions/most_similar.py

** Most similar faces **

The following enables to display an image alongside with most similar images:

- The results are weak, first because of the size of the dataset
- Also, the network can be greatly improved

In [ ]:
def display(img):
    img = img.astype('uint8')
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
interesting_classes = list(filter(lambda x: len(x[1])>4, classid_to_ids.items()))
class_idx = random.choice(interesting_classes)[0]

print(class_idx)
img_idx = random.choice(classid_to_ids[class_idx])
for id, sim in most_sim(emb[img_idx], emb):
    display(all_imgs[id])
    print((classid_to_name[id_to_classid[id]], id, sim))

**Playing with the camera**
- The following code enables you to find the most similar faces to yours
- What do you observe?
- Try to think of reasons why it doesn't work very well, and how you could improve it

In [ ]:
import cv2

def camera_grab(camera_id=0, fallback_filename=None):
    camera = cv2.VideoCapture(camera_id)
    try:
        # take 10 consecutive snapshots to let the camera automatically tune
        # itself and hope that the contrast and lightning of the last snapshot
        # is good enough.
        for i in range(10):
            snapshot_ok, image = camera.read()
        if snapshot_ok:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        else:
            print("WARNING: could not access camera")
            if fallback_filename:
                image = imread(fallback_filename)
    finally:
        camera.release()
    return image

In [ ]:
image = camera_grab(camera_id=0, fallback_filename='laptop.jpeg')
x = resize100(image)
out = shared_conv.predict(np.reshape(x, (1,60,60,3)))
display(x)
for id, sim in most_sim(out[0], emb):
    display(all_imgs[id])
    print((classid_to_name[id_to_classid[id]], id, sim))